# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of *C++* library.

First, import **AutoWIG**.

In [ ]:
import autowig

In [ ]:
autowig.scons('./basic', 'cpp')

In [ ]:
import sys

Assuming that headers are located in the `'./basic'` forlder, we then parse headers with relevant compilation flags.

In [ ]:
asg = autowig.AbstractSemanticGraph()
asg = autowig.parser(asg, [sys.prefix + '/include/basic/overload.h',
                           sys.prefix + '/include/basic/binomial.h'],
                          ['-x', 'c++', '-std=c++11'],
                          silent = True)

Since most of **AutoWIG** guidelines are respected, the `default` `controller` implementation is thus suitable.

In [ ]:
autowig.controller.plugin = 'default'
asg = autowig.controller(asg)

In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [ ]:
autowig.generator.plugin = 'boost_python_internal'
wrappers = autowig.generator(asg, module = 'basic/src/py/_module.cpp',
                                  decorator = 'basic/src/py/basic/_module.py',
                                  prefix = 'wrapper_')

The wrappers are only generated in-memory.
We therefore need to write them on the disk to complete the process.

In [ ]:
wrappers.write()

Once the wrappers are written on disk, we need to compile the *Python* bindings using **SCons**.

In [ ]:
autowig.scons('./basic', 'py')

In [ ]:
import subprocess
subprocess.check_output(['pip', 'install', '-e', './basic'])

In [ ]:
import basic
binomial = basic.BinomialDistribution(1, .5)

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.pmf(1)

In [ ]:
binomial.n = 0

In [ ]:
binomial.pmf(0)

In [ ]:
binomial.set_pi(1.1)